In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

import misc_funcs
import data
import ava_parameter_functions
import power_law_func

import csv

from multiprocess import Pool,cpu_count
from itertools import product


map2_config = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': '92kf1c',
     'type': 'arc',
     'config': {'dataId': '256,320',
      'label': 'new layer',
      'color': [218, 0, 0],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat0': 'lat1',
       'lng0': 'long1',
       'lat1': 'lat2',
       'lng1': 'long2'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 2,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'sizeRange': [0.1, 2],
       'targetColor': None},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': {'name': 'TE', 'type': 'real'},
      'sizeScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'256,320': [{'name': 'long1',
        'format': None},
       {'name': 'lat1', 'format': None},
       {'name': 'long2', 'format': None},
       {'name': 'lat2', 'format': None},
       {'name': 'TE', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': 1.7172966072248994,
   'longitude': 18.101964233149975,
   'pitch': 0,
   'zoom': 2.2866482635409926,
   'isSplit': False},
  'mapStyle': {'styleType': 'light',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': False,
    'road': False,
    'border': True,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

## Generate causal structure for a given dx and time

In [4]:
dx = 640
time = 64

#Do not change the variables below this line. (Pipelines for different conflict types and gridix are not set yet)

conflict_type = "battles"
gridix = 0
polygons = gpd.read_file(f'voronoi_grids/{dx}/borders{str(gridix).zfill(2)}.shp')

In [ ]:
time_series = misc_funcs.time_series_all_polygons(time,dx,conflict_type)
polygons_TE,neighbor_info_dataframe,list_of_tuples_tiles = misc_funcs.neighbor_finder_TE(time_series,time,dx,conflict_type)

Calculating transfer entropy between tiles!
Shuffle Number: 0
Shuffle Number: 10
Shuffle Number: 20
Shuffle Number: 30
Shuffle Number: 40


In [ ]:
list_of_tuples_tiles  #nan means that we have no links between these two polygons

## Visualize the causal network on African map

In [ ]:
tile_pairs = np.array(list_of_tuples_tiles)
tile_pairs = tile_pairs[np.isnan(tile_pairs[:,2]) != True]

pol1 = polygons.loc[tile_pairs[:,0]]["geometry"]
pol2 = polygons.loc[tile_pairs[:,1]]["geometry"]

centroid1 = pol1.centroid.reset_index(drop=True)
centroid2 = pol2.centroid.reset_index(drop=True)

TE_links_data = pd.DataFrame(columns=["one_lng" , "one_lat" , "two_lng" , "two_lat" , "TE"])

one_lng = [list(centroid1[i].coords)[0][0] for i in range(len(centroid1))]
one_lat = [list(centroid1[i].coords)[0][1] for i in range(len(centroid1))]

two_lng = [list(centroid2[i].coords)[0][0] for i in range(len(centroid2))]
two_lat = [list(centroid2[i].coords)[0][1] for i in range(len(centroid2))]

TE_links_data["one_lng"] = one_lng
TE_links_data["one_lat"] = one_lat
TE_links_data["two_lng"] = two_lng
TE_links_data["two_lat"] = two_lat

TE_links_data["TE"] = tile_pairs[:,2]

In [ ]:
from keplergl import KeplerGl
map2=KeplerGl(height=800)

map2.config = map2_config

map2.add_data(data=TE_links_data, name=f"{str(time)},{str(dx)}")

In [ ]:
map2